In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from google.colab import drive
import math
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



1. Create a movie_based_df with the title + year + genre + tag

In [ ]:
# users_based_df = pd.read_csv('/content/drive/MyDrive/CSC413 [Group]/Setup/Data/cleaned/user_based.csv')
movies_df = pd.read_csv('/content/drive/MyDrive/CSC413 [Group]/Setup/Data/cleaned/movies.csv')
ratings_df = pd.read_csv('/content/drive/MyDrive/CSC413 [Group]/Setup/Data/cleaned/ratings.csv')
movies_df = pd.read_csv('/content/drive/MyDrive/CSC413 [Group]/Setup/Data/cleaned/movies.csv')
tags_df = pd.read_csv('/content/drive/MyDrive/CSC413 [Group]/Setup/Data/cleaned/tags.csv')
# links_df = pd.read_csv('/content/drive/MyDrive/CSC413 [Group]/Setup/Data/cleaned/links.csv')
# genome_df = pd.read_csv('/content/drive/MyDrive/CSC413 [Group]/Setup/Data/cleaned/genome.csv')

In [ ]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"['Adventure', 'Animation', 'Children', 'Comedy...",1995.0
1,2,Jumanji,"['Adventure', 'Children', 'Fantasy']",1995.0
2,3,Grumpier Old Men,"['Comedy', 'Romance']",1995.0
3,4,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",1995.0
4,5,Father of the Bride Part II,['Comedy'],1995.0


In [ ]:
# users_based_df.head()

In [ ]:
#
# movie_genome_df = pd.merge(movie_links_df, genome_df, on='movieId', how='left')
movie_ratings_avg = ratings_df.groupby('movieId')['rating'].mean().reset_index()
movie_tags_agg = tags_df.groupby('movieId')['tag'].apply(list).reset_index()
# movie_based_df = pd.merge(movie_genome_df, movie_ratings_avg, on='movieId', how='left')
movie_based_df = pd.merge(movies_df, movie_tags_agg, on='movieId', how='left')

In [ ]:
print(len(movie_based_df))

62423


In [ ]:
# small_movies_df = movie_based_df.sample(n=10000, random_state=100)  # random_state for reproducibility
# small_movies_df.head()

2. Merging all the columns in movie_based_df and return top 5 most likely to be recommended movie by calling cosine similarity



In [ ]:
# combine values of all columns
def create_soup(x):
  genres_str = ' '.join(x['genres']) if isinstance(x['genres'], list) else x['genres']
  tags_str = ' '.join(x['tag']) if isinstance(x['tag'], list) else x['tag']

  # Convert the year to an integer and then to a string if it's not NaN, otherwise use an empty string
  year_str = str(int(x['year'])) if pd.notnull(x['year']) else ''

  return f"{x['title']} {genres_str} {tags_str} {year_str}"


In [ ]:
movie_model = movie_based_df # Create a copy of movie_based_df
movie_model['soup'] = movie_model.apply(create_soup, axis=1)
movie_model.head()

,movieId,title,genres,year,tag,soup
0,1,Toy Story,"['Adventure', 'Animation', 'Children', 'Comedy...",1995.0,"[Owned, imdb top 250, Pixar, Pixar, time trave...","Toy Story ['Adventure', 'Animation', 'Children..."
1,2,Jumanji,"['Adventure', 'Children', 'Fantasy']",1995.0,"[Robin Williams, time travel, fantasy, based o...","Jumanji ['Adventure', 'Children', 'Fantasy'] R..."
2,3,Grumpier Old Men,"['Comedy', 'Romance']",1995.0,"[funny, best friend, duringcreditsstinger, fis...","Grumpier Old Men ['Comedy', 'Romance'] funny b..."
3,4,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",1995.0,"[based on novel or book, chick flick, divorce,...","Waiting to Exhale ['Comedy', 'Drama', 'Romance..."
4,5,Father of the Bride Part II,['Comedy'],1995.0,"[aging, baby, confidence, contraception, daugh...",Father of the Bride Part II ['Comedy'] aging b...


In [ ]:
# Create the TfidfVectorizer object with English stop words
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Compute the TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(movie_model['soup'])

def get_recommendations_on_demand(title, tfidf_matrix, df, tfidf_vectorizer):
    # Transform the 'soup' of the given movie to a TF-IDF vector
    # Note: Use the same TF-IDF vectorizer object used for the dataset
    if title not in df['title'].values:
        print(f"Movie '{title}' not found.")
        return []

    movie_index = df.index[df['title'] == title].tolist()[0]  # Get the index of the movie
    movie_tfidf_vector = tfidf_matrix[movie_index]

    # Calculate the cosine similarity scores between this movie's vector and all movies in the dataset
    cosine_sim = cosine_similarity(movie_tfidf_vector, tfidf_matrix)

    # Get scores and sort by similarity
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the top N similar movies, excluding the first one (itself)
    top_n = 5  # number of recommendations you want
    sim_scores = sim_scores[1:top_n + 1]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    recommendations = []
    for i in movie_indices:
        movie_data = {
            'title': df.iloc[i]['title'],
            'score': sim_scores[movie_indices.index(i)][1],
            'genres': df.iloc[i]['genres'],
            'tags': df.iloc[i]['tag']
        }
        recommendations.append(movie_data)

    return recommendations

    # Fetch the movies details and return them
    # return df.iloc[movie_indices]



In [ ]:
title = 'Toy Story'
recommendations = get_recommendations_on_demand(title, tfidf_matrix, movie_model, tfidf_vectorizer)
for rec in recommendations:
    genres = rec['genres'].split(', ')
    tags = rec['tags']
    print(f"Title: {rec['title']}")
    print(f"Score: {rec['score']:.4f}")
    print(f"Genres: {', '.join(genres)}")
    print(f"Tags: {', '.join(tags)}")
    print("-" * 80)

Title: Toy Story 2
Score: 0.9300
Genres: ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']
Tags: Disney, Owned, imdb top 250, original, animation, Disney, Pixar, bright, DARING RESCUES, fanciful, humorous, light, RACE AGAINST TIME, TOYS COME TO LIFE, whimsical, airplane, collector, duringcreditsstinger, flea market, friendship, garage sale, identity crisis, inanimate objects coming to life, museum, personification, prosecution, rescue team, teamwork, toy comes to life, animation, Disney, funny, original, Pixar, sequel, Tom Hanks, Pixar, original, animation, Pixar, dolls, animation, comedy, Disney, friendship, imdb top 250, Oscar Nominee, Pixar, Tim Allen, Tom Hanks, whimsical, animation, childish, Disney, Pixar, sequel, toys, whimsical, Pixar, toys, animation, Disney, Pixar, animation, Pixar, sequel, Pixar, abandonment, imdb top 250, Pixar, sequel, computer animation, Disney, Pixar, sequel, animation, cute, fanciful, toys, animation, Disney, Pixar, animation, Disney, Disney, 

**DRAFT**

This approach also works but always crashes when running with the full movie_based_df

In [ ]:
# # Create the TfidfVectorizer object with English stop words
# tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# # Compute the TF-IDF matrix
# tfidf_matrix = tfidf_vectorizer.fit_transform(movie_model['soup'])

# # Compute the cosine similarity matrix
# cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# # Reset the index of your DataFrame and construct a reverse mapping from titles to index
# movie_model = movie_model.reset_index()
# indices = pd.Series(movie_model.index, index=movie_model['title'])

# # Recommendation function
# def get_recommendations(title, cosine_sim=cosine_sim, df=movie_model, indices=indices):
#     # Get the index of the movie that matches the title
#     idx = indices[title]

#     # Get the pairwise similarity scores of all movies with that movie and convert it into a list of tuples
#     sim_scores = list(enumerate(cosine_sim[idx]))

#     # Sort the movies based on the similarity scores in descending order
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

#     # Get the scores of the 5 most similar movies, excluding the first one (as it is the movie itself)
#     sim_scores = sim_scores[1:6]

#     # Get the movie indices
#     movie_indices = [i[0] for i in sim_scores]

#     # Fetch the movies details and format the result
#     recommendations = []
#     for i in movie_indices:
#         movie_data = {
#             'title': df.iloc[i]['title'],
#             'score': sim_scores[movie_indices.index(i)][1],
#             'genres': df.iloc[i]['genres'],
#             'tags': df.iloc[i]['tag']
#         }
#         recommendations.append(movie_data)

#     return recommendations

# recommended_movies = get_recommendations('Luz')
# for rec in recommended_movies:
#     genres = rec['genres'].split(', ')
#     print(f"Title: {rec['title']}")
#     print(f"Score: {rec['score']:.4f}")
#     print(f"Genres: {', '.join(genres)}")
#     print(f"Tags: {', '.join(rec['tags'])}")
#     print("-" * 80)

# # print(recommended_movies)